# 链式调用示例

In [2]:
import pandas as pd

data = pd.DataFrame(dict(date=pd.date_range("20220101", "20220201", freq="1W")))
data.head()

,date
0,2022-01-02
1,2022-01-09
2,2022-01-16
3,2022-01-23
4,2022-01-30


In [3]:
data["date"] = data["date"].astype(str)
data["year"] = data["date"].map(lambda s: s.split("-")[0])
data["month"] = data["date"].map(lambda s: s.split("-")[1])
data["day"] = data["date"].map(lambda s: s.split("-")[2])
data["year_month"] = data["year"].str.cat(data["month"])
data.head()

,date,year,month,day,year_month
0,2022-01-02,2022,01,02,202201
1,2022-01-09,2022,01,09,202201
2,2022-01-16,2022,01,16,202201
3,2022-01-23,2022,01,23,202201
4,2022-01-30,2022,01,30,202201


## 第一种重构方式

In [4]:
data = (
    pd.DataFrame({"date": pd.date_range("20220101", "20220201", freq="1W")})
    .astype({"date": "str"})["date"]
    .str.split("-", expand=True)
    .rename(
        columns={
            0: "year",
            1: "month",
            2: "day",
        }
    )
    .assign(
        date=lambda df: df.apply(lambda r: "-".join(r.tolist()), axis=1),
        year_month=lambda df: df["year"].str.cat(df["month"]),
    )
)
data.head()

,year,month,day,date,year_month
0,2022,01,02,2022-01-02,202201
1,2022,01,09,2022-01-09,202201
2,2022,01,16,2022-01-16,202201
3,2022,01,23,2022-01-23,202201
4,2022,01,30,2022-01-30,202201


## 第二种重构方式

In [5]:
data = (
    pd.DataFrame({"date": pd.date_range("20220101", "20220201", freq="1W")})
    .astype({"date": "str"})["date"]
    .str.extract(r"(?P<year>\d{4})-(?P<month>\d{2})-(?P<day>\d{2})")
    .assign(
        date=lambda df: df.apply(lambda row: "-".join(row.tolist()), axis=1),
        year_month=lambda df: df["year"].str.cat(df["month"]),
    )
)

data.head()

,year,month,day,date,year_month
0,2022,01,02,2022-01-02,202201
1,2022,01,09,2022-01-09,202201
2,2022,01,16,2022-01-16,202201
3,2022,01,23,2022-01-23,202201
4,2022,01,30,2022-01-30,202201


## 第三种重构方式

In [6]:
data = (
    pd.DataFrame({"date": pd.date_range("20220101", "20220201", freq="1W")})
    .assign(
        year=lambda df: df["date"].dt.year,
        month=lambda df: df["date"].dt.month.astype(str).str.zfill(2),
        day=lambda df: df["date"].dt.day.astype(str).str.zfill(2),
        year_month=lambda df: df["date"].dt.strftime("%Y%m"),
    )
    .astype(str)
)
data.head()

,date,year,month,day,year_month
0,2022-01-02,2022,01,02,202201
1,2022-01-09,2022,01,09,202201
2,2022-01-16,2022,01,16,202201
3,2022-01-23,2022,01,23,202201
4,2022-01-30,2022,01,30,202201


# 链式调用的实现原理以及优劣

In [7]:
class Command:
    def __init__(self, name=None):
        self.name = name

    def set_options(self, **kwargs):
        for k, v in kwargs.items():
            if not hasattr(self, k):
                setattr(self, k, v)
        return self

    def parse(self):
        opts = [f"{v}" for k, v in self.__dict__.items() if k != "name"]
        final = " ".join([self.name, *opts])
        return final

    def __repr__(self):
        opts = [f"{k}={v}" for k, v in self.__dict__.items() if k != "name"]
        return f"Command<name: {self.name}, options[{', '.join(opts)}]>"

In [8]:
cmd = Command("curl").set_options(verbose="-v").set_options(target="sspai.com")

In [9]:
print(cmd)

Command<name: curl, options[verbose=-v, target=sspai.com]>


In [10]:
print(cmd.parse())

curl -v sspai.com
